In [14]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import pandas as pd

def carregar_dados_excel():
    df = pd.read_excel('Cidades_Expandido.xlsx',sheet_name='Dados')
    return df

df = carregar_dados_excel()

def atualizar_listboxes(event=None):
    pais_selecionado = [listbox_pais.get(i) for i in listbox_pais.curselection()]
    
    estados_filtrados = df[df['Pais'].isin(pais_selecionado)]['Estado'].unique()

    listbox_estado.delete(0,tk.END)

    for estado in sorted(estados_filtrados):
        listbox_estado.insert(tk.END,estado)

    listbox_cidade.delete(0,tk.END)

    atualizar_treeview()

def atualizar_treeview(event=None):
    pais_selecionados = [
        listbox_pais.get(i) for i in listbox_pais.curselection()
    ]

    estados_selecionados = [
        listbox_estado.get(i) for i in listbox_estado.curselection()
    ]

    cidades_selecionadas = [
        listbox_cidade.get(i) for i in listbox_cidade.curselection()
    ]

    dados_filtrados = df

    if pais_selecionados:
        dados_filtrados = dados_filtrados[dados_filtrados['Pais'].isin(pais_selecionados)]

    if estados_selecionados:
        dados_filtrados = dados_filtrados[dados_filtrados['Estado'].isin(estados_selecionados)]
    
    if cidades_selecionadas:
        dados_filtrados = dados_filtrados[dados_filtrados['Cidade'].isin(cidades_selecionadas)]

    for item in tree.get_children():
        tree.delete(item)

    for index,row in dados_filtrados.iterrows():
        tree.insert('',tk.END,
                    values=(row['Pais'],row['Estado'],row['Cidade']))        
        
def atualizar_cidades(event=None):
    estado_selecionados = [listbox_estado.get(i) for i in listbox_estado.curselection()]

    cidades_filtradas = df[df['Estado'].isin(estado_selecionados)]['Cidade'].unique()

    listbox_cidade.delete(0,tk.END)

    for cidade in sorted(cidades_filtradas):
        listbox_cidade.insert(tk.END,cidade)

    atualizar_treeview()

def exportar_para_excel():
    pais_selecionado = [listbox_pais.get(i) for i in listbox_pais.curselection()]
    estado_selecionado = [listbox_estado.get(i) for i in listbox_estado.curselection()]
    cidade_selecionado = [listbox_cidade.get(i) for i in listbox_cidade.curselection()]

    dados_filtrados = df

    if pais_selecionado:
        dados_filtrados = dados_filtrados[dados_filtrados['Pais'].isin(pais_selecionado)]
        
    if estado_selecionado:
        dados_filtrados = dados_filtrados[dados_filtrados['Estado'].isin(estado_selecionado)]
        
    if cidade_selecionado:
        dados_filtrados = dados_filtrados[dados_filtrados['Cidade'].isin(cidade_selecionado)]

    caminho_arquivo = filedialog.asksaveasfilename(defaultextension='.xlsx',
                                                   filetype=[('Excel files','*.xlsx')])
    
    if caminho_arquivo:
        dados_filtrados.to_excel(caminho_arquivo,index=False)

janela = tk.Tk()
janela.title("Seleção Múltipla com Listbox")
janela.geometry("800x600")

frame_listbox = tk.Frame(janela)
frame_listbox.pack(padx=10,pady=10)

label_pais = tk.Label(frame_listbox, text="País: ")
label_pais.grid(row=0,column=0,padx=5)

listbox_pais = tk.Listbox(frame_listbox,
                          selectmode=tk.MULTIPLE,
                          exportselection=False,
                          width=20,
                          height=10)
listbox_pais.grid(row=1,column=0,padx=5)

label_estado = tk.Label(frame_listbox, text="Estado: ")
label_estado.grid(row=0,column=1,padx=5)

listbox_estado = tk.Listbox(frame_listbox,
                          selectmode=tk.MULTIPLE,
                          exportselection=False,
                          width=20,
                          height=10)
listbox_estado.grid(row=1,column=1,padx=5)

label_cidade = tk.Label(frame_listbox, text="Cidade: ")
label_cidade.grid(row=0,column=2,padx=5)

listbox_cidade = tk.Listbox(frame_listbox,
                          selectmode=tk.MULTIPLE,
                          exportselection=False,
                          width=20,
                          height=10)
listbox_cidade.grid(row=1,column=2,padx=5)

listbox_pais.bind("<<ListboxSelect>>",atualizar_listboxes)
listbox_estado.bind("<<ListboxSelect>>",atualizar_cidades)
listbox_cidade.bind("<<ListboxSelect>>",atualizar_treeview)

tree = ttk.Treeview(janela,
                    columns=('Pais','Estado','Cidade'),
                    show='headings')
tree.heading("Pais",text='Pais')
tree.heading("Estado",text='Estado')
tree.heading("Cidade",text='Cidade')
tree.pack(expand=True,fill=tk.BOTH)

btn_exportar = tk.Button(janela,
                         text='Exportar para Excel',
                         command=exportar_para_excel)
btn_exportar.pack(pady=10)

for pais in sorted(df['Pais'].unique()):
    listbox_pais.insert(tk.END,pais)

janela.mainloop()